**Importación de librerías requeridas**

In [1]:
import pandas as pd
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By

**Obtención de la información a través de Google Maps**

In [2]:
# Creación del driver

driver = webdriver.Chrome(ChromeDriverManager().install())

# Link de Google Maps con la busqueda especificada

page_url = "https://www.google.com.ec/maps/search/Sweet+%26+Coffee+en+Guayaquil/@-2.1832438,-79.9698488,13z/data=!3m1!4b1?hl=es"

# Obtención del link

driver.get(page_url)

# Agrandado de pantalla

driver.maximize_window()

# Obtención de cada Sweet & Coffee de Guayaquil

time.sleep(2)

map_div = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')

# Scroll down hasta cargar toda la información disponible

for i in range(23):
    driver.execute_script("arguments[0].scrollBy(0, 1000);", map_div)
    time.sleep(3)

time.sleep(2)

# Extracción del HTML de los resultados de la búsqueda

soup = BeautifulSoup(driver.page_source, "html.parser")

# Encontrar todos los elementos de los resultados de la búsqueda

items = driver.find_elements(By.CLASS_NAME, "hfpxzc")

# Obtención de cada uno de los Sweet & Coffee disponibles en GYE (nombre y url que contiene la dirección)

sweets = []

for item in items:
    sweet_url = item.get_attribute("href")
    sweet_name = item.get_attribute("aria-label")
    sweets.append({"sweet_name": sweet_name, "url": sweet_url })

# Obtención de las direcciones
    
full_info = []

for sweet in sweets:
    if sweet == sweets[-1]:
        break
    else:
        driver.get(sweet["url"])
        data_address = driver.find_element(By.XPATH, "//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[3]/button/div[1]/div[2]/div[1]")
        info_address = data_address.text
        data_rating = driver.find_element(By.XPATH, "//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[1]/span/span[1]")
        info_rating = data_rating.text
        full_info.append({"sweet_name": sweet["sweet_name"], "address": info_address ,"rating": info_rating})

C:\Users\PC\AppData\Local\Temp\ipykernel_13820\2630589973.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[1]/span/span[1]"}
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x009037D3]
	(No symbol) [0x00898B81]
	(No symbol) [0x0079B36D]
	(No symbol) [0x007CD382]
	(No symbol) [0x007CD4BB]
	(No symbol) [0x00803302]
	(No symbol) [0x007EB464]
	(No symbol) [0x00801215]
	(No symbol) [0x007EB216]
	(No symbol) [0x007C0D97]
	(No symbol) [0x007C253D]
	GetHandleVerifier [0x00B7ABF2+2510930]
	GetHandleVerifier [0x00BA8EC1+2700065]
	GetHandleVerifier [0x00BAC86C+2714828]
	GetHandleVerifier [0x009B3480+645344]
	(No symbol) [0x008A0FD2]
	(No symbol) [0x008A6C68]
	(No symbol) [0x008A6D4B]
	(No symbol) [0x008B0D6B]
	BaseThreadInitThunk [0x7737F989+25]
	RtlGetAppContainerNamedObjectPath [0x77AF74B4+228]
	RtlGetAppContainerNamedObjectPath [0x77AF7484+180]


**Extracción de información para limpieza posterior**

In [3]:
# Creación del dataframe con la información requerida

list_of_dicts = []

for inf in full_info:
    name = inf["sweet_name"]
    zip_code = inf["address"][-6:]
    location = inf["address"][:-6]
    list_of_dicts.append({"name": name, "location": location, "zip_code": zip_code , "rating": inf["rating"]})
    
df = pd.DataFrame(list_of_dicts)

In [4]:
df

,name,location,zip_code,rating
0,Sweet & Coffee - Primax 25 Julio,Av. 25 de Julio y Ernesto Albán Mosquera Prima...,090102,"4,7"
1,Sweet & Coffee - Urdesa,"C. 9 NO, Guayaquil",090511,"4,7"
2,Sweet & Coffee - 9 de Octubre,"Av. 9 de Octubre y, Av. José de Antepara, Guay...",090311,"4,6"
3,Sweet & Coffee - Bahía Malecón,Malecón Simón Bolívar entre Sucre y Cristobal ...,090313,"4,6"
4,Sweet & Coffee,Guayaquil,090513,"4,6"
...,...,...,...,...
62,Sweet and coffee,"R4G2+HF2, Av del Periodista, Guayaquil",090512,"5,0"
63,Sweet & Coffee - Primax Nexus,"Autop. Narcisa de Jesús, Guayaquil",090703,"4,8"
64,Sweet & Coffee • Plaza Lagos,Km. 6.5 Vía a La Aurora Plaza Lagos Town Cente...,092302,"4,7"
65,Sweet & Coffee - Primax Equitransa,"Av León Febres Cordero Ribadeneyra, Guayaquil",091910,"5,0"


**Guardado de información**

In [5]:
# Creación del csv que será utilizado para la visualización de la data

df.to_csv("sweet_&_coffee_data_set_new.csv",index=False)

**Finalización del código**

In [6]:
# Finalización del driver

driver.quit

<bound method ChromiumDriver.quit of <selenium.webdriver.chrome.webdriver.WebDriver (session="ff20ebd90561ac31727cf3b311bb162a")>>